In [17]:
from lean_dojo import LeanGitRepo, trace

repo = LeanGitRepo("https://github.com/yangky11/lean4-example", "7b6ecb9ad4829e4e73600a3329baeb3b5df8d23f")
trace(repo, dst_dir="traced_lean4-example")

AssertionError: The destination directory traced_lean4-example already exists.

In [19]:
from lean_dojo import *

repo = LeanGitRepo("https://github.com/yangky11/lean4-example", "7b6ecb9ad4829e4e73600a3329baeb3b5df8d23f")
theorem = Theorem(repo, "Lean4Example.lean", "hello_world")

with Dojo(theorem) as (dojo, init_state):
  print(init_state)
  result = dojo.run_tac(init_state, "rw [add_assoc, add_comm b, ←add_assoc]")
  assert isinstance(result, ProofFinished)
  print(result)

TacticState(pp='a b c : Nat\n⊢ a + b + c = a + c + b', id=0, message=None)
ProofFinished(tactic_state_id=1, message='')


In [42]:
import os
import subprocess

def lean_compile(state: str):
    temp_file_path = "./hello_world.lean"
    with open(temp_file_path, "w") as f:
        f.write(state)
    result = subprocess.run(['lean', temp_file_path], capture_output=True, text=True)
    if result.returncode == 0:
        return False
    else:
        return result.stdout

def lean_compile(state: str):
    # Get the current working directory (where your Lake project is)
    project_dir = os.getcwd()
    temp_file_path = os.path.join(project_dir, "hello_world.lean")
    
    with open(temp_file_path, "w") as f:
        f.write(state)
    
    # First make sure we're in the Lake project directory
    os.chdir(project_dir)
    
    # Try using lake env to run lean
    result = subprocess.run(
        ['lake', 'env', 'lean', temp_file_path],
        capture_output=True,
        text=True
    )
    
    if result.returncode == 0:
        return False
    else:
        return result.stdout

In [58]:
lean_compile("""
import Mathlib.Data.Real.Basic
import Mathlib.Data.Nat.choose.sum
import Mathlib.Data.Nat.Sqrt
--  add add_pow
   

theorem squares (a b : ℝ) : (a + b) ^ 2 = a ^ 2 + 2 * a * b + b ^ 2
:= by
  rw [pow_two]
""")

'/Users/arnavmehta/Desktop/AIInEd/LeanProjectTest/hello_world.lean:9:3: error: unsolved goals\na b : ℝ\n⊢ (a + b) * (a + b) = a ^ 2 + 2 * a * b + b ^ 2\n'

In [75]:
import os
import json
from openai import OpenAI


# index = 41 <- zero element kernel theorem

def ask_llm(statement):
    client = OpenAI(api_key='sk-32g4iVVXfcUcLKNLpsTSG7SpgHE0CsWjrbnv7rnnaqT3BlbkFJa3l7JCjuz4hAJGiEErZhj82FYufDqQdycFldWzWxEA')
    prompt = (
        f"""
        Here are some examples before we start:

        Example 1:
        	
        NL: A: $ x_1,x_2,...,x_n \ge 0$ B: $ x_1^2+x_2^2+...+x_n^2 \ge 0$
        Lean: theorem lean_workbook_plus_522 (n : ℕ) (x : ℕ → ℝ) : ∀ i : ℕ, i ≤ n → x i ≥ 0 → ∑ i in Finset.range n, (x i)^2 ≥ 0 := by sorry
        We are writing a theorem to prove that A \implies B: where
        A: {statement["statement1"]}
        B: {statement["statement2"]}
        return the theorem with lean formalism and sorry for the tactic 
        Make sure you return in Lean 4 code that compiles 
        DO NO PROVIDE ANY TACTICS JUST WRITE sorry
        GIVE ME ONLY THE LEAN 4 CODE SO I CAN DIRECTLY COMPILE IT
        MAKE SURE YOU USE Mathlib4
        HAVE NO IMPORT STATEMENTS
        """
        )
    response = client.chat.completions.create(messages=[{"role": "user", "content": prompt}], model="gpt-4o-mini", temperature=0, n=1)             
    return(response.choices[0].message.content)

def ask_llm_again(statement, LLMoutput, error):
    client = OpenAI(api_key='sk-32g4iVVXfcUcLKNLpsTSG7SpgHE0CsWjrbnv7rnnaqT3BlbkFJa3l7JCjuz4hAJGiEErZhj82FYufDqQdycFldWzWxEA')
    prompt = (
        f"""

        Here are some examples before we start:

        Example 1:
        	
        NL: A: $ x_1,x_2,...,x_n \ge 0$ B: $ x_1^2+x_2^2+...+x_n^2 \ge 0$
        Lean: theorem lean_workbook_plus_522 (n : ℕ) (x : ℕ → ℝ) : ∀ i : ℕ, i ≤ n → x i ≥ 0 → ∑ i in Finset.range n, (x i)^2 ≥ 0 := by sorry

        We are writing a theorem to prove that A \implies B: where
        A: {statement["statement1"]}
        B: {statement["statement2"]}
        return the theorem with lean formalism and sorry for the tactic 
        Make sure you return in Lean 4 code that compiles 
        DO NO PROVIDE ANY TACTICS JUST WRITE sorry
        this was your output the last time you tried to compile: {LLMoutput}
        and this was the error: {error}
        GIVE ME ONLY THE LEAN 4 CODE SO I CAN DIRECTLY COMPILE IT
        MAKE SURE YOU USE Mathlib4
        HAVE NO IMPORT STATEMENTS
        """
        )
    response = client.chat.completions.create(messages=[{"role": "user", "content": prompt}], model="gpt-4o-mini", temperature=0, n=1)             
    return(response.choices[0].message.content)

def process_llm_output(output):

    return "import Mathlib \n" + output.split("```")[1][5:]

def LLM_autoformal(statement: str):
    response = process_llm_output(ask_llm(statement))
    
    if lean_compile(response):
        print("FAILURE", statement, response, lean_compile(response))
        response_again = process_llm_output(ask_llm_again(statement, response, lean_compile(response)))
        print("AGAIN RESPONSE", statement, response_again, lean_compile(response_again))
    else: 
        print("Success", statement, response)
    print("HELLO WE ARE DONE")


In [76]:
LLM_autoformal({
    "statement1": "$a_1, a_2, a_3, \dots a_n \in \mathbb{N}$ are in arithmetic progression",
   "statement2": "$\sum_{i=0}^n a_i = \frac{a_1}{a_2}$",
})

Success {'statement1': '$a_1, a_2, a_3, \\dots a_n \\in \\mathbb{N}$ are in arithmetic progression', 'statement2': '$\\sum_{i=0}^n a_i = \x0crac{a_1}{a_2}$'} import Mathlib 
theorem lean_workbook_plus_523 (n : ℕ) (a : ℕ → ℝ) (h : ∀ i : ℕ, i < n → a i + 1 = a i + d) : ∑ i in Finset.range n, a i = (n * (a 0 + a (n - 1))) / 2 := by sorry

HELLO WE ARE DONE


In [35]:
import os
import subprocess

# Define your theorem as a string
state = 'theorem hello_world (a b c : Nat): a + b + c = a + c + b := by sorry'

# Write the string to a temporary .lean file
temp_file_path = "./tmp/hello_world.lean"
os.makedirs("./tmp", exist_ok=True)
with open(temp_file_path, "w") as f:
    f.write(state)

# Run lean on the file
result = subprocess.run(['lean', temp_file_path], capture_output=True, text=True)

# Check if there were any errors
if result.returncode == 0:
    print("Theorem is valid!")
else:
    print(f"Errors found:\n{result.stdout}")
# print(result)

Theorem is valid!


In [57]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("kaiyuy/leandojo-lean4-tacgen-byt5-small")
model = AutoModelForSeq2SeqLM.from_pretrained("kaiyuy/leandojo-lean4-tacgen-byt5-small")

# state = "n : ℕ\n⊢ gcd n n = n"
# state = "arnav"
state = "(a b : ℝ) :(a + b) ^ 2 = a ^ 2 + 2 * a * b + b ^ 2"
# state: "give me the tactic that goes from state (a+b)^2 = (a+b)*(a+b) to a^2 + 2*a*b + b^2"
tokenized_state = tokenizer(state, return_tensors="pt")

# Generate a single tactic.
tactic_ids = model.generate(tokenized_state.input_ids, max_length=1024)
tactic = tokenizer.decode(tactic_ids[0], skip_special_tokens=True)
print(tactic, end="\n\n")

# Generate multiple tactics via beam search.
tactic_candidates_ids = model.generate(
    tokenized_state.input_ids,
    max_length=1024,
    num_beams=4,
    length_penalty=0.0,
    do_sample=False,
    num_return_sequences=4,
    early_stopping=False,
)
tactic_candidates = tokenizer.batch_decode(
    tactic_candidates_ids, skip_special_tokens=True
)
for tac in tactic_candidates:
    print(tac)

/usr/local/lib/python3.11/site-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
/usr/local/lib/python3.11/site-packages/torchvision/transforms/v2/__init__.py:54: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https:/

rw [← sq_sqrt, ← sq_sqrt, ← sq_sqrt, ← sq_sqrt, ← sq_sqrt, ← sq_sqrt, ← sq_sqrt]

rw [add_comm]
simp only [add_pow]
simp [add_pow, add_mul]
simp only [add_pow, add_pow, mul_pow, mul_pow]


In [ ]:
# tidy, reprover, LLM_step